In [13]:
from pathlib import Path
import polars as pl
import json
import numpy as np
# 数据加载和基础预处理
path = Path(r'C:\Users\matrix\Desktop\kaggle\芒果tv\Data_A')
 # 'did_features'
 # 'vid_info'
 # '弹幕文本数据',
 # '用户历史播放数据'
 # '用户历史曝光数据'
 # '用户历史点击数据'
 # 'A榜待预测的did'
 # 'A榜用户曝光数据'
def read_log_data(data_type='view',day=1,read_type='read'):
    
    
    data_path = path 
    
    # data_path = path
    
    if day<10:
        date = '0'+str(day)
    else:
        date = str(day)
    
    if data_type == 'view':
        data_path = data_path / '用户历史曝光数据'/ f'day{date}' / f'did_show_data{date}.csv'
    elif data_type == 'click':
         data_path = data_path /'用户历史点击数据'/ f'day{date}' / f'day{date}_data.csv'
    elif data_type == 'play':
        data_path = data_path /'用户历史播放数据'/ f'day{date}' / f'day{date}_data.csv'
    else:
        raise NameError(f'no such data type "{data_type}"')
        
    if read_type=='read':
        return pl.read_csv(data_path)
    elif read_type=='scan':
        return pl.scan_csv(data_path)
    else:
        raise NameError(f'no such read type "{data_type}"')

def scan_log_data(start_day,end_day):
    scan_data_all  = {'view':{},'click':{},'play':{}}

    for day_i in range(start_day,end_day):
        scan_data_all['view'][day_i] =  read_log_data(data_type='view',day=day_i,read_type='scan')
        scan_data_all['click'][day_i] =  read_log_data(data_type='click',day=day_i,read_type='scan')
        scan_data_all['play'][day_i] =  read_log_data(data_type='play',day=day_i,read_type='scan')
    return scan_data_all
def load_log_data(start_day,end_day):
    data_all  = {'view':{},'click':{},'play':{}}

    for day_i in range(start_day,end_day):
        data_all['view'][day_i] =  read_log_data(data_type='view',day=day_i,read_type='read')
        data_all['click'][day_i] =  read_log_data(data_type='click',day=day_i,read_type='read')
        data_all['play'][day_i] =  read_log_data(data_type='play',day=day_i,read_type='read')
    return data_all

In [14]:
def process_user_data():
    user_data = pl.read_csv(path/'did_features/did_features_table.csv')
    (path/'mappings').mkdir(exist_ok=True)
    
    cat_columns = [
        col for col in user_data.columns 
        if (col not in ('did', 'f87')) and (user_data[col].n_unique() < len(user_data)*0.3 and (user_data[col] == user_data[col].cast(pl.Int64)).all())
    ]
    num_columns = [col for col in user_data.columns if col not in cat_columns and col != 'did']
    # 填充枚举类特征
    if cat_columns:
        user_data = user_data.with_columns([
            pl.col(col).fill_null(-1.0).cast(pl.Int64)
            for col in cat_columns
        ])
    cat_columns.append('f87')
    
    mapping_dicts = {}
    for col in cat_columns:
        # 生成映射字典
        unique_values = user_data[col].unique().to_list()
        mapping = {v: i for i, v in enumerate(unique_values)}
        mapping_dicts[col] = mapping
        
        # 应用映射并保存
        user_data = user_data.with_columns(
            pl.col(col).map_elements(lambda x: mapping.get(x, -1), return_dtype=pl.Int64).alias(col)
        )
        with open(path/f'mappings/{col}_mapping.json', 'w') as f:
            json.dump(mapping, f)

    # 填充数值类特征
    if num_columns:
        user_data = user_data.with_columns([
            pl.col(col).fill_null(pl.col(col).mean()).cast(pl.Float32)  # 填充均值并转换为Float32类型
            for col in num_columns
        ]
    )
    norm_params = {}
    for col in num_columns:
        # 计算均值和标准差
        mean_val = user_data[col].mean()
        std_val = user_data[col].std()
        norm_params[col] = {'mean': mean_val, 'std': std_val}
        
        # 应用标准化
        user_data = user_data.with_columns(
            ((pl.col(col) - mean_val) / std_val).alias(col)
        )
    # 保存正则化参数
    with open(path/'mappings/normalization_params.json', 'w') as f:
        json.dump(norm_params, f)

    return user_data

def process_vid_data():
    vid_data = pl.read_csv(path/'vid_info/vid_info_table.csv')

    (path/'mappings').mkdir(exist_ok=True)
    num_columns = ["item_duration"]
    cat_columns = [col for col in vid_data.columns if col not in num_columns and col != 'vid']
    if cat_columns:
        vid_data = vid_data.with_columns([
            pl.col(col).fill_null(-1.0).cast(pl.Int64)
            for col in cat_columns
        ])
    mapping_dicts = {}
    
    for col in cat_columns:
        # 生成映射字典
        unique_values = vid_data[col].unique().to_list()
        mapping = {v: i for i, v in enumerate(unique_values)}
        mapping_dicts[col] = mapping
        
        # 应用映射并保存
        vid_data = vid_data.with_columns(
            pl.col(col).map_elements(lambda x: mapping.get(x, -1), return_dtype=pl.Int64).alias(col)
        )
        with open(path/f'mappings/{col}_mapping.json', 'w') as f:
            json.dump(mapping, f)
    norm_params = {}
    for col in num_columns:
        # 计算均值和标准差
        mean_val = vid_data[col].mean()
        std_val = vid_data[col].std()
        norm_params[col] = {'mean': mean_val, 'std': std_val}
        
        # 应用标准化
        vid_data = vid_data.with_columns(
            ((pl.col(col) - mean_val) / std_val).cast(pl.Float32).alias(col+'_n')
        )
    # 保存正则化参数
    param_file = path/'mappings/normalization_params.json'
    if param_file.exists():
        with open(param_file, 'r') as f:
            existing_params = json.load(f)
    
    # 更新参数（新参数覆盖旧值）
    existing_params.update(norm_params)
    
    # 保存合并后的参数
    with open(param_file, 'w') as f:
        json.dump(existing_params, f, indent=4)
    return vid_data



In [15]:
# 构造数据集
def create_dataset(start_day=1, end_day=31):
    load_data = load_log_data(start_day,end_day)
    # 处理用户特征
    user_data = process_user_data()
    # 处理视频特征
    vid_data = process_vid_data()
    test_user_data = pl.read_csv(path/"A榜待预测的did/testA_pred_did.csv")
    test_user_show_data = pl.read_csv(path/"A榜用户曝光数据/testA_did_show.csv")

    # 构造点击，播放标签 did,vid,click,play
    train_data = {}
    for day_i in range(start_day,end_day):
        view_click_play_data = load_data['view'][day_i].join(load_data['click'][day_i],on=['did','vid'],how='left').join(load_data['play'][day_i],on=['did','vid'],how='left')
        train_data_label = view_click_play_data.with_columns(
            # pl.when(pl.col('click_time').is_not_null()).then(1).otherwise(0).alias('click'),
            pl.col('click_time').is_not_null().alias('click'),
            pl.col('play_time').fill_null(pl.lit(0)).alias('play_time')
        )
        train_data[day_i] = train_data_label.select(['did','vid','click','play_time'])
    history = []
    for i in range(start_day,end_day):
        print(train_data[i].shape)
        tmp = train_data[i].join(vid_data, on="vid").join(user_data, on="did")
        tmp = tmp.with_columns(
            pl.lit(i).alias("day"),
            (pl.col("play_time")/pl.col("item_duration")).alias("play_ratio"),
            ).join(test_user_data, on="did", how="semi")
        history.append(tmp)  

    test_data = test_user_show_data.join(vid_data, on="vid").join(user_data, on="did")
    test_data = test_data.with_columns(
            pl.lit(0).alias("click"),
            pl.lit(0).alias("play_time"),
            pl.lit(32).alias("day"),
            pl.lit(0).alias("play_ratio"),
            )
    train_data = pl.concat(history)
    return train_data, test_data

In [16]:
df,test = create_dataset(1,31)


(1492712, 4)
(1346235, 4)
(1089715, 4)
(1363623, 4)
(1212882, 4)
(1041831, 4)
(1260247, 4)
(1167650, 4)
(889703, 4)
(727634, 4)
(981486, 4)
(1416403, 4)
(874771, 4)
(956684, 4)
(1025837, 4)
(764269, 4)
(767565, 4)
(812498, 4)
(760550, 4)
(1345987, 4)
(1718881, 4)
(1232513, 4)
(934105, 4)
(1006762, 4)
(1099988, 4)
(833238, 4)
(1029554, 4)
(1147831, 4)
(846025, 4)
(757846, 4)


In [17]:
df_train = df.filter((pl.col('day')<=29) & (pl.col('day')>=1)).to_pandas()
df_valid = df.filter(pl.col('day')==30).to_pandas()
# df_test = test.to_pandas()

df_train.to_parquet('ctr_df_train_add.parquet',index=False)
df_valid.to_parquet('ctr_df_valid_add.parquet',index=False)
# df_test.to_parquet('ctr_test.parquet',index=False)

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3991734 entries, 0 to 3991733
Columns: 104 entries, did to play_ratio
dtypes: bool(1), float32(38), float64(1), int32(1), int64(62), object(1)
memory usage: 2.5+ GB
